In [4]:
import os
import pandas as pd
import seaborn as sns

In [5]:
dir_name = "datasets"
target = pd.read_csv(f"{dir_name}/D_target.csv")
salary = pd.read_csv(f"{dir_name}/D_salary.csv")
loan = pd.read_csv(f"{dir_name}/D_loan.csv")
close_loan = pd.read_csv(f"{dir_name}/D_close_loan.csv")
clients = pd.read_csv(f"{dir_name}/D_clients.csv")

In [6]:
close_loan = loan.merge(close_loan)\
.drop(columns=["ID_LOAN"])\
.groupby("ID_CLIENT", as_index=False)\
.sum()\
.rename(columns={"CLOSED_FL": "LOAN_NUM_CLOSED"})

loan = loan.groupby("ID_CLIENT", as_index=False).count().rename(columns={"ID_LOAN": "LOAN_NUM_TOTAL"})

clients.rename(columns={"ID": "ID_CLIENT"}, inplace=True)
clients.drop(columns=[
    "EDUCATION", "MARITAL_STATUS", "REG_ADDRESS_PROVINCE", "FACT_ADDRESS_PROVINCE", 
    "POSTAL_ADDRESS_PROVINCE", "FL_PRESENCE_FL", "OWN_AUTO"
], inplace=True)

clients = clients.merge(loan, how = "left", on="ID_CLIENT")
clients = clients.merge(close_loan, how = "left", on="ID_CLIENT")

salary.drop(columns=["FAMILY_INCOME"], inplace=True)
salary = salary.groupby("ID_CLIENT", as_index=False).mean()
clients = clients.merge(salary, how = "left", on="ID_CLIENT")

target.drop(columns=["AGREEMENT_RK"], inplace=True)
clients = clients.merge(target, how = "left", on = "ID_CLIENT")

clients.rename(columns={"ID_CLIENT": "AGREEMENT_RK"}, inplace=True)
clients = clients.fillna(clients.median())

clients = clients.applymap(round)

clients.head()

,AGREEMENT_RK,AGE,GENDER,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,PERSONAL_INCOME,TARGET
0,106805103,42,1,1,0,1,0,1,0,25000,1
1,106809308,28,1,1,1,1,0,1,0,10000,0
2,106805867,64,0,2,0,1,1,1,1,30000,0
3,106808779,54,1,0,0,1,0,1,0,25000,0
4,106814289,26,0,1,1,1,0,2,2,15000,0


In [7]:
clients.to_csv("dataset.csv", index=False)